# <B> Setup for Anormaly Detection with SageMaker </B>
* Container: codna_python3

## AutoReload

In [25]:
%load_ext autoreload
%autoreload 2

import sys, os
module_path = ".."
sys.path.append(os.path.abspath(module_path))

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## 0. Install packages

In [2]:
install_needed = True  # should only be True once
# install_needed = False

In [3]:
%%bash
#!/bin/bash

DAEMON_PATH="/etc/docker"
MEMORY_SIZE=10G

FLAG=$(cat $DAEMON_PATH/daemon.json | jq 'has("data-root")')
# echo $FLAG

if [ "$FLAG" == true ]; then
    echo "Already revised"
else
    echo "Add data-root and default-shm-size=$MEMORY_SIZE"
    sudo cp $DAEMON_PATH/daemon.json $DAEMON_PATH/daemon.json.bak
    sudo cat $DAEMON_PATH/daemon.json.bak | jq '. += {"data-root":"/home/ec2-user/SageMaker/.container/docker","default-shm-size":"'$MEMORY_SIZE'"}' | sudo tee $DAEMON_PATH/daemon.json > /dev/null
    sudo service docker restart
    echo "Docker Restart"
fi

Already revised


In [4]:
import sys
import IPython

if install_needed:
    print("installing deps and restarting kernel")
    !{sys.executable} -m pip install -U pip
    !{sys.executable} -m pip install -U awscli
    !{sys.executable} -m pip install -U botocore
    !{sys.executable} -m pip install -U boto3
    !{sys.executable} -m pip install -U sagemaker 
    !{sys.executable} -m pip install -U awswrangler
    !{sys.executable} -m pip install -U jsonpath_ng
    
    

    IPython.Application.instance().kernel.do_shutdown(True)

installing deps and restarting kernel


## 1. Set roles

In [2]:
from sagemaker import get_execution_role

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


In [3]:
strSageMakerRoleName = get_execution_role().rsplit('/', 1)[-1]
print (f"SageMaker Execution Role Name: {strSageMakerRoleName}")

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml
SageMaker Execution Role Name: AmazonSageMaker-ExecutionRole-20221206T163436


## 1.1 Attach IAM polich to sagemaker execution role (<b>with console</b>)
> **S3**: "arn:aws:iam::aws:policy/AmazonS3FullAccess"

### 1.1 Attach IAM polich to sagemaker execution role (<b>with console</b>)
> step 1. IAM console 로 이동    
>
> step 2. 왼쪽 네비게이터에서  "Role" 선택  
> ![nn](../imgs/Role.png)  
>
> step 3. SageMaker Execution Role 검색 후 role 선택 (상위 cell output 참조)  
> ![nn](../imgs/search-by-rolename.png)  
>
> step 4. "attach policies" 메뉴 선택 
> ![nn](../imgs/attach-policy-menu.png)  
>
> step 5. "IAMFullAccess" policy 검색 후 attach 
> ![nn](../imgs/attach-policy.png) 

In [6]:
from utils.iam import iam_handler

In [7]:
iam = iam_handler()

### 1.2 Attach policy - sagemaker execution role
> SSM  
> S3 <BR>
> Kinesis <BR>
> KinesisAnalytics <BR>

In [37]:
listPolicyArn=[
    "arn:aws:iam::aws:policy/AmazonSSMFullAccess",
    "arn:aws:iam::aws:policy/AmazonS3FullAccess",
    "arn:aws:iam::aws:policy/AWSGlueConsoleSageMakerNotebookFullAccess"
]
for strPolicyArn in listPolicyArn: iam.attach_policy(strSageMakerRoleName, strPolicyArn)

## 2. Set default parameters

In [88]:
import boto3
import sagemaker

### Bucket / Prefix 설정

In [89]:
strRegionName = boto3.Session().region_name
strAccountId = boto3.client("sts").get_caller_identity().get("Account")
strBucketName = 'sm-anomaly-detection' # <-- 사용할 bucket 명을 추가해 주세요. ex) sagemaker-us-east-1-123456789123, sm-nemo-bucket
strPrefix = 'ad' ## <-- 작업할 prefix 명을 추가해 주세요. ex) nemo-test, nemo-asr

## 3. Extract & Upload dataset

In [90]:
import os
from utils.s3 import s3_handler

In [91]:
strLocalDataPath = "./data"

* create bucket

In [92]:
s3 = s3_handler(region_name="us-east-1")

This is a S3 handler with [us-east-1] region.


In [93]:
s3.create_bucket(strBucketName)

CREATE:[sm-anomaly-detection] Bucket was created successfully


True

- (if needed) delete_bucket

In [94]:
#s3.delete_bucket(strBucketName)

* updoad data to s3

In [95]:
source_dir, target_bucket, target_dir = strLocalDataPath, strBucketName, "data"
s3.upload_dir(source_dir, target_bucket, target_dir)

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml
Upload:[./data] was uploaded to [s3://sm-anomaly-detection/data]successfully


## 4. [Optional] AWS Systems Manager Parameter Store 를 이용한 파라미터 저장/활용
- [AWS Systems Manager Parameter Store](https://docs.aws.amazon.com/systems-manager/latest/userguide/systems-manager-parameter-store.html)
- Attach IAM polich to sagemaker execution role (<b>with console</b>)
> **SSM**: "arn:aws:iam::aws:policy/AmazonSSMFullAccess"<BR>

In [19]:
from utils.ssm import parameter_store

In [20]:
pm = parameter_store(strRegionName)

In [21]:
pm.put_params(key="PREFIX", value=strPrefix, overwrite=True)
pm.put_params(key="-".join([strPrefix, "REGION"]), value=strRegionName, overwrite=True)
pm.put_params(key="-".join([strPrefix, "BUCKET"]), value=strBucketName, overwrite=True)
pm.put_params(key="-".join([strPrefix, "SAGEMAKER-ROLE-ARN"]), value=get_execution_role(), overwrite=True)
pm.put_params(key="-".join([strPrefix, "ACCOUNT-ID"]), value=strAccountId, overwrite=True)
pm.put_params(key="-".join([strPrefix, "S3-DATA-PATH"]), value=f"s3://{strBucketName}/data", overwrite=True)

'Store suceess'

In [22]:
print (f'PREFIX: {pm.get_params(key="PREFIX")}')
print (f'REGION: {pm.get_params(key="-".join([strPrefix, "REGION"]))}')
print (f'BUCKET: {pm.get_params(key="-".join([strPrefix, "BUCKET"]))}')
print (f'SAGEMAKER-ROLE-ARN: {pm.get_params(key="-".join([strPrefix, "SAGEMAKER-ROLE-ARN"]))}')
print (f'ACCOUNT-ID: {pm.get_params(key="-".join([strPrefix, "ACCOUNT-ID"]))}')
print (f'S3-DATA-PATH: {pm.get_params(key="-".join([strPrefix, "S3-DATA-PATH"]))}')

PREFIX: ad-ts
REGION: us-east-1
BUCKET: sm-anomaly-detection
SAGEMAKER-ROLE-ARN: arn:aws:iam::419974056037:role/service-role/AmazonSageMaker-ExecutionRole-20221206T163436
ACCOUNT-ID: 419974056037
S3-DATA-PATH: s3://sm-anomaly-detection/data
